# Import the required Libraries

In [0]:
pip install geohash2

Python interpreter will be restarted.
  Created wheel for geohash2: filename=geohash2-1.1-py3-none-any.whl size=15559 sha256=430f21cbd9b3790d23e729191f44c52fbfcf6f5eb932650984e9e1f48f37c22e
  Stored in directory: /root/.cache/pip/wheels/ec/29/ab/aa2b0265ceaef06eac4aed2ca6ae1b17eb2ffb14d611f20172
Successfully built geohash2
Python interpreter will be restarted.


In [0]:
pip install pygeohash

Python interpreter will be restarted.
  Created wheel for pygeohash: filename=pygeohash-1.2.0-py2.py3-none-any.whl size=6166 sha256=07eefb6687f1d9a580ae430da74f2c467ad2f34b094e254b29f9b378887b9b26
  Stored in directory: /root/.cache/pip/wheels/6e/e7/a2/4ea56930e1cc3ece852306b842f9ca71e0cd9482cb8b848e27
Successfully built pygeohash
Python interpreter will be restarted.


In [0]:
pip install shapely

Python interpreter will be restarted.
Python interpreter will be restarted.


In [0]:
pip install pykdtree

In [0]:
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Geohash").getOrCreate()
import numpy as np
from pyspark.sql.functions import *
import os
import sys
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number
from functools import reduce
from pyspark.sql import DataFrame
import pyspark
#sc=pyspark.SparkContext()
from pyspark.context import SparkContext
from pyspark.sql import SparkSession
from pyspark.sql import HiveContext
#import sys
from pyspark.sql.window import Window
from pyspark.sql import functions as F
from pyspark.sql.functions import lead, lag, col ,datediff, to_date, lit, size , length , year, month, dayofmonth, from_unixtime, unix_timestamp, desc, asc, rank, min , expr, trim, date_format, avg
import datetime
from datetime import datetime
from pyspark.sql.functions import regexp_replace, col
from pyspark.sql.types import IntegerType, DoubleType, StringType, DateType, TimestampType, StructField, NumericType
from pyspark.sql.functions import isnan
from pyspark.sql.functions import udf
from io import BytesIO
#import boto3
import math
from pyspark.sql import Row
#import pyarrow as pa
import pandas as pd
from datetime import datetime, timedelta
#from time import gmtime, strftime ,datetime
from pyspark.sql.functions import concat, col, lit
from pyspark.sql.functions import count, min,max
from pyspark.sql.functions import col, when
spark.sql("set spark.sql.legacy.timeParserPolicy=LEGACY")
from pyspark.sql.functions import col, row_number
import geohash2 as pgh
import pygeohash as pgh2
from scipy.spatial import cKDTree
from shapely.geometry import Point

# Read the dataset 

In [0]:
df = spark.read.csv("/FileStore/tables/vehicle_dataset-1.csv",header=True,inferSchema=True)

In [0]:
df.show()

+---------+------------------+------------------+----------+----------+-----------+-----------------+-----------+------------+-----------------+----------+
|vehicleID|       gpsLatitude|      gpsLongitude| eventdate|       LOB|ProductLine|ParentProductLINE|Rashturning|Overspeeding|Harshacceleration|Harshbrake|
+---------+------------------+------------------+----------+----------+-----------+-----------------+-----------+------------+-----------------+----------+
|     7271|   86.631841050332|129.25898534880483|2022-04-15|  Personal|   ProductB|          ParentC|          3|           5|                3|         3|
|      861| 57.44859144264761| 38.87756176747753|2022-06-26|  Personal|   ProductC|          ParentB|          5|           5|                4|         3|
|     5391|50.079960172342815| 46.29365669442484|2022-06-03|  Personal|   ProductA|          ParentA|          0|           4|                0|         3|
|     5192| 79.92239779367982| 135.4795766850667|2022-05-22|  Pe

## Geohash > Encode

In [0]:
udf1 = F.udf(lambda x,y: pgh.encode(x,y,precision=7)) # Check counts with 5 as well
df1= df.select('vehicleID','eventdate','gpsLatitude','gpslongitude','LOB','ProductLine','ParentProductLine',
                'rashturning','overspeeding','harshacceleration','harshbrake',
                udf1('gpsLatitude','gpslongitude').alias('geohash_7'))

In [0]:
df1.show()

+---------+----------+------------------+------------------+----------+-----------+-----------------+-----------+------------+-----------------+----------+---------+
|vehicleID| eventdate|       gpsLatitude|      gpslongitude|       LOB|ProductLine|ParentProductLine|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|
+---------+----------+------------------+------------------+----------+-----------+-----------------+-----------+------------+-----------------+----------+---------+
|     7271|2022-04-15|   86.631841050332|129.25898534880483|  Personal|   ProductB|          ParentC|          3|           5|                3|         3|  yz7uv6g|
|      861|2022-06-26| 57.44859144264761| 38.87756176747753|  Personal|   ProductC|          ParentB|          5|           5|                4|         3|  uf5wudj|
|     5391|2022-06-03|50.079960172342815| 46.29365669442484|  Personal|   ProductA|          ParentA|          0|           4|                0|         3|  v0buvky|
|   

In [0]:
udf2 = F.udf(lambda x,y: pgh.encode(x,y,precision=5)) # Check counts with 5 as well
df1= df1.select('vehicleID','eventdate','gpsLatitude','gpslongitude','LOB','ProductLine','ParentProductLine',
                'rashturning','overspeeding','harshacceleration','harshbrake','geohash_7',
                udf2('gpsLatitude','gpslongitude').alias('geohash_5'))


In [0]:
 df1.show(3)

+---------+----------+------------------+------------------+--------+-----------+-----------------+-----------+------------+-----------------+----------+---------+---------+
|vehicleID| eventdate|       gpsLatitude|      gpslongitude|     LOB|ProductLine|ParentProductLine|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|geohash_5|
+---------+----------+------------------+------------------+--------+-----------+-----------------+-----------+------------+-----------------+----------+---------+---------+
|     7271|2022-04-15|   86.631841050332|129.25898534880483|Personal|   ProductB|          ParentC|          3|           5|                3|         3|  yz7uv6g|    yz7uv|
|      861|2022-06-26| 57.44859144264761| 38.87756176747753|Personal|   ProductC|          ParentB|          5|           5|                4|         3|  uf5wudj|    uf5wu|
|     5391|2022-06-03|50.079960172342815| 46.29365669442484|Personal|   ProductA|          ParentA|          0|           4|      

## Decode Geohash

In [0]:
# UDF to decode geohash_7
import pygeohash as pgh2
from pyspark.sql.functions import pandas_udf, col
from pyspark.sql.types import ArrayType, DoubleType,FloatType
udf2 = F.udf(lambda x: pgh2.decode(x),ArrayType(FloatType()))

df2 = df1.select('vehicleID','eventdate','gpsLatitude','gpslongitude','LOB','ParentProductLine','ProductLine',
                 'rashturning','overspeeding','harshacceleration','harshbrake','geohash_7',
                 'geohash_5',udf2('geohash_7').alias('decodedValue'))




df2.show()

+---------+----------+------------------+------------------+----------+-----------------+-----------+-----------+------------+-----------------+----------+---------+---------+---------------+
|vehicleID| eventdate|       gpsLatitude|      gpslongitude|       LOB|ParentProductLine|ProductLine|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|geohash_5|   decodedValue|
+---------+----------+------------------+------------------+----------+-----------------+-----------+-----------+------------+-----------------+----------+---------+---------+---------------+
|     7271|2022-04-15|   86.631841050332|129.25898534880483|  Personal|          ParentC|   ProductB|          3|           5|                3|         3|  yz7uv6g|    yz7uv|[86.63, 129.26]|
|      861|2022-06-26| 57.44859144264761| 38.87756176747753|  Personal|          ParentB|   ProductC|          5|           5|                4|         3|  uf5wudj|    uf5wu| [57.45, 38.88]|
|     5391|2022-06-03|50.079960172342815

In [0]:
from pyspark.sql.functions import split
from pyspark.sql.functions import col
# split the decoded value into latitude and longitude columns
df3 = df2.withColumn("Latitude", col("decodedValue")[0].cast("float"))
df3 = df3.withColumn("Longitude", col("decodedValue")[1].cast("float"))
df3.show(3)


+---------+----------+------------------+------------------+--------+-----------------+-----------+-----------+------------+-----------------+----------+---------+---------+---------------+--------+---------+
|vehicleID| eventdate|       gpsLatitude|      gpslongitude|     LOB|ParentProductLine|ProductLine|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|geohash_5|   decodedValue|Latitude|Longitude|
+---------+----------+------------------+------------------+--------+-----------------+-----------+-----------+------------+-----------------+----------+---------+---------+---------------+--------+---------+
|     7271|2022-04-15|   86.631841050332|129.25898534880483|Personal|          ParentC|   ProductB|          3|           5|                3|         3|  yz7uv6g|    yz7uv|[86.63, 129.26]|   86.63|   129.26|
|      861|2022-06-26| 57.44859144264761| 38.87756176747753|Personal|          ParentB|   ProductC|          5|           5|                4|         3|  uf5wudj| 

In [0]:
df4= df3.drop("decodedValue")

In [0]:
df4.show()

+---------+----------+------------------+------------------+----------+-----------------+-----------+-----------+------------+-----------------+----------+---------+---------+--------+---------+
|vehicleID| eventdate|       gpsLatitude|      gpslongitude|       LOB|ParentProductLine|ProductLine|rashturning|overspeeding|harshacceleration|harshbrake|geohash_7|geohash_5|Latitude|Longitude|
+---------+----------+------------------+------------------+----------+-----------------+-----------+-----------+------------+-----------------+----------+---------+---------+--------+---------+
|     7271|2022-04-15|   86.631841050332|129.25898534880483|  Personal|          ParentC|   ProductB|          3|           5|                3|         3|  yz7uv6g|    yz7uv|   86.63|   129.26|
|      861|2022-06-26| 57.44859144264761| 38.87756176747753|  Personal|          ParentB|   ProductC|          5|           5|                4|         3|  uf5wudj|    uf5wu|   57.45|    38.88|
|     5391|2022-06-03|50.

In [0]:
df5=df4.select('vehicleId','geohash_7','Latitude', 'Longitude','eventDate','LOB','ProductLine','ParentProductLine','rashturning','overspeeding','harshacceleration','harshbrake').groupBy('geohash_7','vehicleId','Latitude', 'Longitude','eventDate','LOB','ProductLine','ParentProductLine','rashturning','overspeeding','harshacceleration','harshbrake').agg(countDistinct('vehicleId'))

In [0]:
df5.show()

+---------+---------+--------+---------+----------+----------+-----------+-----------------+-----------+------------+-----------------+----------+----------------+
|geohash_7|vehicleId|Latitude|Longitude| eventDate|       LOB|ProductLine|ParentProductLine|rashturning|overspeeding|harshacceleration|harshbrake|count(vehicleId)|
+---------+---------+--------+---------+----------+----------+-----------+-----------------+-----------+------------+-----------------+----------+----------------+
|  wnbujje|     1307|    38.7|    91.28|2022-11-14|Commercial|   ProductA|          ParentC|          0|           2|                0|         2|               1|
|  t66ud5r|     7281|   13.47|    60.22|2022-06-09|  Personal|   ProductC|          ParentA|          5|           0|                1|         2|               1|
|  zr1wptz|      570|   85.46|    148.7|2022-11-01|  Personal|   ProductB|          ParentC|          0|           3|                5|         4|               1|
|  wf073gc|     

In [0]:
df5 = df5.withColumnRenamed("count(vehicleId)", "VinCount")


In [0]:
df5.columns

Out[20]: ['geohash_7',
 'vehicleId',
 'Latitude',
 'Longitude',
 'eventDate',
 'LOB',
 'ProductLine',
 'ParentProductLine',
 'rashturning',
 'overspeeding',
 'harshacceleration',
 'harshbrake',
 'VinCount']

In [0]:
print(df.count())
print(df5.count())

150000
150000


In [0]:
from pyspark.sql.functions import col, sum as _sum
# Filter the original DataFrame to create four separate DataFrames
rashturning_df = df5.filter((col("rashturning") > 0) & (col("overspeeding") == 0) & (col("harshacceleration") == 0) & (col("harshbrake") == 0))
overspeeding_df = df5.filter((col("rashturning") == 0) & (col("overspeeding") > 0) & (col("harshacceleration") == 0) & (col("harshbrake") == 0))
harshacceleration_df = df5.filter((col("rashturning") == 0) & (col("overspeeding") == 0) & (col("harshacceleration") > 0) & (col("harshbrake") == 0))
harshbrake_df = df5.filter((col("rashturning") == 0) & (col("overspeeding") == 0) & (col("harshacceleration") == 0) & (col("harshbrake") > 0))

# Aggregate VinCount in a temporary DataFrame
vincount_df = df5.groupBy("vehicleID").agg(_sum("VinCount").alias("totalVinCount"))
vincount_df = vincount_df.filter(col("totalVinCount") > 10)

# Join the filtered DataFrames with the aggregated VinCount DataFrame
rashturning_df = rashturning_df.join(vincount_df, "vehicleID")
overspeeding_df = overspeeding_df.join(vincount_df, "vehicleID")
harshacceleration_df = harshacceleration_df.join(vincount_df, "vehicleID")
harshbrake_df = harshbrake_df.join(vincount_df, "vehicleID")


In [0]:
print("Overspeeding : ", overspeeding_df.count())
print("harshacceleration : ", harshacceleration_df.count())
print("harshbrake : ", overspeeding_df.count())
print("rashturning : ", rashturning_df.count())

Overspeeding :  507
harshacceleration :  562
harshbrake :  507
rashturning :  555


### Reverse Geocode 

from pyspark.sql import SparkSession
from pyspark.sql.functions import udf, struct, col
from pyspark.sql.types import StructType, StructField, StringType
from geopy.geocoders import Nominatim

# Create a SparkSession
spark = SparkSession.builder.getOrCreate()

# Sample DataFrame (replace this with your actual DataFrame)
data = [
    (1, "2024-01-30", 37.7749, -122.4194, "LOB1", "ParentPL1", "PL1", 1, 0, 0, 0),
    (2, "2024-01-31", 34.0522, -118.2437, "LOB2", "ParentPL2", "PL2", 0, 1, 0, 1),
    # Add more rows as needed
]

schema = ["vehicleID", "eventdate", "gpsLatitude", "gpsLongitude", "LOB", "ParentProductLine", "ProductLine", "rashturning", "overspeeding", "harshacceleration", "harshbrake"]

df = spark.createDataFrame(data, schema=schema)

# Define the UDF for reverse geocoding using geopy
geolocator = Nominatim(user_agent="reverse_geocoding_example")

def reverse_geocode(latitude, longitude):
    location = geolocator.reverse((latitude, longitude))
    address = location.address.split(', ')
    return {
        'City_Town': address[0],
        'State': address[-3],
        'District': address[-4]
    }

# Register the UDF with the return type as struct
reverse_geocode_udf = udf(reverse_geocode, StructType([
    StructField('City_Town', StringType(), nullable=False),
    StructField('State', StringType(), nullable=False),
    StructField('District', StringType(), nullable=False)
]))

# Add the UDF to the DataFrame and extract individual fields
df_with_geocoded_data = df.withColumn(
    'data', reverse_geocode_udf('gpsLatitude', 'gpsLongitude')
).withColumn(
    'City_Town', col('data').getField('City_Town')
).withColumn(
    'State', col('data').getField('State')
).withColumn(
    'District', col('data').getField('District')
)

# Drop the unnecessary columns
df_with_geocoded_data = df_with_geocoded_data.drop('data')

# Show the resulting DataFrame
df_with_geocoded_data.show(truncate=False)


## ckdTree

In [0]:
# Importing worldcities dataset 

In [0]:
world_cities = spark.read.csv("/FileStore/tables/worldcities.csv",header=True,inferSchema=True)

In [0]:
world_cities.show(3)

+-------+----------+-------+--------+---------+----+----+----------+-------+----------+----------+
|   city|city_ascii|    lat|     lng|  country|iso2|iso3|admin_name|capital|population|        id|
+-------+----------+-------+--------+---------+----+----+----------+-------+----------+----------+
|  Tokyo|     Tokyo|35.6839|139.7744|    Japan|  JP| JPN|     Tōkyō|primary|  3.9105E7|1392685764|
|Jakarta|   Jakarta|-6.2146|106.8451|Indonesia|  ID| IDN|   Jakarta|primary|  3.5362E7|1360771077|
|  Delhi|     Delhi|28.6667| 77.2167|    India|  IN| IND|     Delhi|  admin|   3.187E7|1356872604|
+-------+----------+-------+--------+---------+----+----+----------+-------+----------+----------+
only showing top 3 rows



In [0]:
world_cities = world_cities.filter(col("country") == 'India')

In [0]:
world_cities.count()

Out[28]: 1600

In [0]:
world_cities.show()

+--------------+--------------+-------+-------+-------+----+----+--------------+-------+----------+----------+
|          city|    city_ascii|    lat|    lng|country|iso2|iso3|    admin_name|capital|population|        id|
+--------------+--------------+-------+-------+-------+----+----+--------------+-------+----------+----------+
|         Delhi|         Delhi|28.6667|77.2167|  India|  IN| IND|         Delhi|  admin|   3.187E7|1356872604|
|        Mumbai|        Mumbai|19.0758|72.8775|  India|  IN| IND|   Mahārāshtra|  admin|  2.2186E7|1356226629|
|       Kolkāta|       Kolkata|22.5727|88.3639|  India|  IN| IND|   West Bengal|  admin|  1.8698E7|1356060520|
|     Bangalore|     Bangalore|12.9791|77.5913|  India|  IN| IND|     Karnātaka|  admin|  1.3999E7|1356410365|
|       Chennai|       Chennai|13.0825| 80.275|  India|  IN| IND|    Tamil Nādu|  admin|  1.1564E7|1356374944|
|     Hyderābād|     Hyderabad|17.3617|78.4747|  India|  IN| IND|     Telangana|  admin| 9840000.0|1356871768|
|

In [0]:
def create_geometry(lat, lng):
    return Point(float(lng), float(lat))

create_geometry_udf = udf(create_geometry)

In [0]:
df_ind = world_cities.select("city_ascii", "lat", "lng")

In [0]:
df_ind.show()

+--------------+-------+-------+
|    city_ascii|    lat|    lng|
+--------------+-------+-------+
|         Delhi|28.6667|77.2167|
|        Mumbai|19.0758|72.8775|
|       Kolkata|22.5727|88.3639|
|     Bangalore|12.9791|77.5913|
|       Chennai|13.0825| 80.275|
|     Hyderabad|17.3617|78.4747|
|          Pune|18.5196|73.8553|
|     Ahmedabad|  23.03|  72.58|
|     Allahabad|  25.45|  81.85|
|         Surat|  21.17|  72.83|
|       Lucknow| 26.847| 80.947|
|        Jaipur|26.9167|75.8667|
|      Cawnpore|26.4725|80.3311|
|      Mirzapur|  25.15|  82.58|
|        Nagpur|21.1539|79.0831|
|     Ghaziabad|28.6667|77.4167|
|      Vadodara|   22.3|   73.2|
|Vishakhapatnam|17.7333|83.3167|
|        Indore|22.7206|75.8472|
|         Thane|  19.18|72.9633|
+--------------+-------+-------+
only showing top 20 rows



In [0]:
from pyspark.sql.functions import col
from pyspark.sql import Row
from pyspark.sql import SparkSession
from scipy.spatial import cKDTree

# Collect the latitude, longitude, and city_ascii columns from df2
df2_coords = df_ind.select("lat", "lng", "city_ascii").collect()

# Create KDTree for df2 coordinates
kdtree = cKDTree([(row["lat"], row["lng"]) for row in df2_coords])

In [0]:
# Testing with sample data 

In [0]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, FloatType
# Define the schema for the DataFrame
schema = StructType([
    StructField("Latitude", FloatType(), True),
    StructField("Longitude", FloatType(), True)
])

# Sample data
data = [
    (28.6139, 77.2090),  # New Delhi
    (19.0760, 72.8777),  # Mumbai
    (13.0827, 80.2707),  # Chennai
    (17.3850, 78.4867),  # Hyderabad
    (12.9716, 77.5946)   # Bangalore
]

# Create the DataFrame
data = spark.createDataFrame(data, schema=schema)

# Show the result
data.show(truncate=False)


+--------+---------+
|Latitude|Longitude|
+--------+---------+
|28.6139 |77.209   |
|19.076  |72.8777  |
|13.0827 |80.2707  |
|17.385  |78.4867  |
|12.9716 |77.5946  |
+--------+---------+



In [0]:
data.columns

Out[44]: ['Latitude', 'Longitude']

In [0]:
# Function to find the nearest city from df2 for each row in df1
def find_nearest_city(row):
    lat = row["gpsLatitude"]
    lng = row["gpsLongitude"]
    # Query the KDTree to find the index of the nearest city in df2
    index = kdtree.query([lat, lng])[1]
    # Get the corresponding city name from df2_coords
    city_name = df2_coords[index]["city_ascii"]
    return city_name

# Convert df1 to RDD of Rows and apply the find_nearest_city function
df1_rdd = data.rdd.map(lambda row: Row(nearest_city=find_nearest_city(row), **row.asDict()))

# Create a new DataFrame from the RDD
df1_with_nearest_city = df1_rdd.toDF()

# Show or perform further operations on df1_with_nearest_city
df1_with_nearest_city.show()


+------------+------------------+-----------------+
|nearest_city|          Latitude|        Longitude|
+------------+------------------+-----------------+
|   New Delhi| 28.61389923095703|77.20899963378906|
|      Mumbai|19.076000213623047|72.87770080566406|
|     Chennai|  13.0826997756958|80.27069854736328|
|   Hyderabad|17.385000228881836|78.48670196533203|
|   Bangalore|12.971599578857422|77.59459686279297|
+------------+------------------+-----------------+



In [0]:
# Testing with HCV CARGO data 

In [0]:
df_HCVcargo = spark.read.csv("/FileStore/tables/MAT1267.csv",header=True,inferSchema=True)

In [0]:
def find_nearest_city(row):
    lat = row["gpsLatitude"]
    lng = row["gpsLongitude"]
    # Query the KDTree to find the index of the nearest city in df2
    index = kdtree.query([lat, lng])[1]
    # Get the corresponding city name from df2_coords
    city_name = df2_coords[index]["city_ascii"]
    return city_name

# Convert df1 to RDD of Rows and apply the find_nearest_city function
df1_rdd = df_HCVcargo.rdd.map(lambda row: Row(nearest_city=find_nearest_city(row), **row.asDict()))

# Create a new DataFrame from the RDD
df1_with_nearest_city = df1_rdd.toDF()

# Show or perform further operations on df1_with_nearest_city
df1_with_nearest_city.show()

+------------+---+----------+-------------+------+------+------+---------+---------+-------------------+----+------------------+------+-----------+------------+----------------+-----+-----+-----+----------+----------------+-------------+--------+--------+--------+-----------------+-------------------+----------+------------------+-------------------+------------------+-----------------+------------------+-----------------------+-----------------------+------------------------+-----------------------+----------------------+---------------------+---------------+--------------+---------+--------+--------------------+--------------------+
|nearest_city|_c0|Unnamed: 0|   receivedat|accelX|accelY|accelZ|analogIp1|analogIp2|      eventDateTime|fuel|gpsCourseInDegrees|gpsFix|gpsLatitude|gpsLongitude|gpsSignalQuality|gyroX|gyroY|gyroZ|ignitionOn|     messageType|noOfSatForFix|odometer|   speed|tenantId|        vehicleId|     eventDateTimeT| eventdate|eventDateTimeTYear|eventDateTimeTMonth|event

In [0]:
df1_with_nearest_city.columns

Out[52]: ['nearest_city',
 '_c0',
 'Unnamed: 0',
 'receivedat',
 'accelX',
 'accelY',
 'accelZ',
 'analogIp1',
 'analogIp2',
 'eventDateTime',
 'fuel',
 'gpsCourseInDegrees',
 'gpsFix',
 'gpsLatitude',
 'gpsLongitude',
 'gpsSignalQuality',
 'gyroX',
 'gyroY',
 'gyroZ',
 'ignitionOn',
 'messageType',
 'noOfSatForFix',
 'odometer',
 'speed',
 'tenantId',
 'vehicleId',
 'eventDateTimeT',
 'eventdate',
 'eventDateTimeTYear',
 'eventDateTimeTMonth',
 'eventDateTimeTWeek',
 'eventDateTimeTDay',
 'eventDateTimeThour',
 'eventDateTimeTDayofweek',
 'eventDateTimeTDayofyear',
 'eventDateTimeTweekofyear',
 'eventDateTimeTMonthName',
 'eventDateTimeTMonthKey',
 'eventDateTimeTWeekKey',
 'Time_diff',
 'City/Town_Name',
 'State',
 'District',
 'geom',
 'address']

In [0]:
df1_with_nearest_city.select('vehicleId','gpsLatitude','gpsLongitude','nearest_city').show(50)

+-----------------+-----------+------------+------------+
|        vehicleId|gpsLatitude|gpsLongitude|nearest_city|
+-----------------+-----------+------------+------------+
|MAT794002MAB01267|  26.962048|  76.7123968|       Alwar|
|MAT794002MAB01267| 26.9612928|  76.7055424|       Alwar|
|MAT794002MAB01267| 26.9604416|  76.6986752|       Alwar|
|MAT794002MAB01267|  26.959632|    76.69184|       Alwar|
|MAT794002MAB01267|   26.95872|  76.6845696|       Alwar|
|MAT794002MAB01267| 26.9579744|  76.6774976|       Alwar|
|MAT794002MAB01267| 26.9571712|  76.6699904|       Alwar|
|MAT794002MAB01267|    26.9564|  76.6630528|       Alwar|
|MAT794002MAB01267| 26.9555616|   76.655616|       Alwar|
|MAT794002MAB01267| 26.9547456|  76.6483328|       Alwar|
|MAT794002MAB01267| 26.9539744|  76.6410112|       Alwar|
|MAT794002MAB01267| 26.9531872|  76.6337024|       Alwar|
|MAT794002MAB01267| 26.9524928|  76.6263104|       Alwar|
|MAT794002MAB01267|   26.95176|  76.6187456|       Alwar|
|MAT794002MAB0